In [2]:
import gym
import pybullet_envs
import pybullet

import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune.registry import register_env
import time
from scipy.spatial.transform import Rotation as R
import pandas as pd
import numpy as np

from low_level_env import LowLevelHumanoidEnv
from hier_env import HierarchicalHumanoidEnv

from pymo.parsers import BVHParser
from pymo.preprocessing import *

In [3]:
def make_env(env_config):
    import pybullet_envs
    return HierarchicalHumanoidEnv()


ENV = 'HumanoidBulletEnv-v0-Hier'
register_env(ENV, make_env)
TARGET_REWARD = 2000

In [4]:
parser = BVHParser()
parsed_data = parser.parse('Dataset/CMU_Mocap_BVH/08/08_01.bvh')
mp = MocapParameterizer('position')
bvh_pos = mp.fit_transform([parsed_data])[0].values

In [5]:
bvh_pos

,Hips_Xposition,Hips_Yposition,Hips_Zposition,LowerBack_Xposition,LowerBack_Yposition,LowerBack_Zposition,Spine_Xposition,Spine_Yposition,Spine_Zposition,Spine1_Xposition,...,LeftLeg_Zposition,LeftFoot_Xposition,LeftFoot_Yposition,LeftFoot_Zposition,LeftToeBase_Xposition,LeftToeBase_Yposition,LeftToeBase_Zposition,LeftToeBase_Nub_Xposition,LeftToeBase_Nub_Yposition,LeftToeBase_Nub_Zposition
0 days 00:00:00,7.1998,15.3951,-37.2754,7.1998,15.3951,-37.2754,7.209830,17.400860,-37.418070,7.218910,...,-36.399380,8.662147,-0.185983,-36.399380,8.655494,-0.566878,-34.429580,8.655494,-0.566878,-33.416880
0 days 00:00:00.008333300,7.1998,15.3951,-37.2754,7.1998,15.3951,-37.2754,7.241606,17.400484,-37.133219,7.274167,...,-35.951809,7.915722,0.608913,-39.139306,7.648375,-0.005262,-37.248129,7.512256,-0.328603,-36.298137
0 days 00:00:00.016666600,7.1954,15.3894,-37.0435,7.1954,15.3894,-37.0435,7.250246,17.394608,-36.903279,7.298927,...,-35.919873,7.922986,0.628395,-39.130001,7.617998,-0.008251,-37.251996,7.454602,-0.441359,-36.351285
0 days 00:00:00.024999900,7.1905,15.3745,-36.8040,7.1905,15.3745,-36.8040,7.255434,17.379674,-36.667660,7.317149,...,-35.883560,7.940175,0.638956,-39.123536,7.696464,0.002640,-37.236490,7.590921,-0.144527,-36.240115
0 days 00:00:00.033333200,7.1935,15.3672,-36.5645,7.1935,15.3672,-36.5645,7.264889,17.372640,-36.435493,7.330707,...,-35.836634,7.959456,0.651586,-39.117457,7.713964,0.001262,-37.235424,7.614706,-0.090904,-36.231823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:00:02.274990900,7.2256,16.1408,26.9953,7.2256,16.1408,26.9953,7.191831,18.149988,27.069811,7.084194,...,25.706442,8.167404,3.260247,20.336059,8.150685,1.357677,20.972600,8.137880,0.379781,21.235502
0 days 00:00:02.283324200,7.2354,16.1494,27.2323,7.2354,16.1494,27.2323,7.197743,18.158559,27.305715,7.074245,...,26.158171,8.213478,3.388311,20.626627,8.229184,1.441191,21.110068,8.234341,0.450145,21.318301
0 days 00:00:02.291657500,7.2360,16.1650,27.4599,7.2360,16.1650,27.4599,7.185947,18.174995,27.429177,7.049802,...,26.592371,8.264271,3.514026,20.919426,8.320775,1.536207,21.251508,8.349773,0.538876,21.424858
0 days 00:00:02.299990800,7.2319,16.1959,27.6841,7.2319,16.1959,27.6841,7.187086,18.205949,27.719058,7.048889,...,27.017796,8.295847,3.652337,21.223626,8.400185,1.657992,21.415823,8.461191,0.664399,21.601870


In [7]:
def getNormalJointPos(df, joint, frame):
    x = df.iloc[frame][joint + '_Xposition']
    y = df.iloc[frame][joint + '_Yposition']
    z = df.iloc[frame][joint + '_Zposition']
    return np.array([x, y, z])

def _getJointPos(df, joint, frame):
    x = df.iloc[frame][joint + '_Xposition']
    z = df.iloc[frame][joint + '_Zposition']
    y = df.iloc[frame][joint + '_Yposition']
    return np.array([z, x, y])/15

def getJointPos(df, joint, frame):
    xStart, yStart, zStart = _getJointPos(df, 'Hips', 1)
    return _getJointPos(df, joint, frame) - np.array([xStart, yStart, 0.01])

def getVectorFrom2Joint(df, j1, j2, frame):
    return getJointPos(df, j2, frame) - getJointPos(df, j1, frame)

In [8]:
endpoint = ['LeftLeg', 'LeftFoot', 'RightLeg', 'RightFoot']
normalized_data = np.zeros((1, 12))
for i in range(1, len(bvh_pos)):
    basePos = _getJointPos(bvh_pos, 'Hips', i)
    tmp = []
    for ep in endpoint:
        epPos = _getJointPos(bvh_pos, ep, i)
        tmp = np.hstack((tmp, epPos - basePos))
    normalized_data = np.vstack((normalized_data, tmp))
normalized_data = normalized_data[1:]

In [9]:
norm_df = pd.DataFrame(normalized_data, columns=['{}_{}position'.format(ep, axis) for ep in endpoint for axis in ['X', 'Y', 'Z']])

In [10]:
norm_df.head()

,LeftLeg_Xposition,LeftLeg_Yposition,LeftLeg_Zposition,LeftFoot_Xposition,LeftFoot_Yposition,LeftFoot_Zposition,RightLeg_Xposition,RightLeg_Yposition,RightLeg_Zposition,RightFoot_Xposition,RightFoot_Yposition,RightFoot_Zposition
0,0.088239,0.078685,-0.559161,-0.124260,0.047728,-0.985746,0.298626,-0.094182,-0.520820,0.084469,-0.048277,-0.947002
1,0.074908,0.080529,-0.558316,-0.139100,0.048506,-0.984067,0.304994,-0.091677,-0.518478,0.114122,-0.045435,-0.955550
2,0.061363,0.083043,-0.557705,-0.154636,0.049978,-0.982370,0.310303,-0.089263,-0.516289,0.144955,-0.042385,-0.963575
3,0.048524,0.084536,-0.557804,-0.170197,0.051064,-0.981041,0.314956,-0.087188,-0.513701,0.176789,-0.040134,-0.970098
4,0.037088,0.085340,-0.559117,-0.183678,0.051349,-0.981249,0.317215,-0.085837,-0.511406,0.207684,-0.038491,-0.975478


In [11]:
# norm_df.to_csv('Joints CSV/walk8_1JointVecFromHip.csv', index=False)

In [12]:
getNormalJointPos(norm_df, 'LeftLeg', 0)

array([ 0.08823941,  0.07868522, -0.55916118])

In [13]:
getVectorFrom2Joint(bvh_pos, 'Hips', 'LeftLeg', 0)

array([ 0.05840133,  0.10582506, -0.56122503])

In [14]:
def quatFromJoints(joints, idx, joint_name):
    x, y, z, w = joints.iloc[idx][joint_name + '.X'], joints.iloc[idx][joint_name +
                                                   '.Y'], joints.iloc[idx][joint_name + '.Z'], joints.iloc[idx][joint_name + '.W']
    return R.from_quat([x, y, z, w]).as_quat().tolist()

def eulerFromJoints(joints, idx, joint_name, order='xyz'):
    # return pybullet.getEulerFromQuaternion(quatFromJoints(joints, idx, joint_name))
    x, y, z = [np.deg2rad(joints.iloc[idx][joint_name+o]) for o in ['.X', '.Y', '.Z']]
    # return R.from_euler('zyx', [z, y, x]).as_euler(order)
    return [x, y, z]

In [15]:
def rotFrom2Vec(v1, v2):
    rotAxis= np.cross(v1, v2)
    rotAxis = rotAxis / np.linalg.norm(rotAxis)
    rotAngle = v1.dot(v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    rotAngle = np.arccos(rotAngle)
    if(rotAxis.all() == 0):
        rotAxis = np.array([1, 0, 0])
        if((v1 == v2).all()):
            rotAngle = 0
        else:
            rotAngle = np.pi
    return R.from_rotvec(rotAxis * rotAngle)

In [16]:
v1 = np.array([3, 3, 21])
v2 = np.array([2, 1, 5])
r = rotFrom2Vec(v1, v2)
print(r.as_euler('xyz', degrees=True))
print(r.apply(v1) / np.linalg.norm(v1) * np.linalg.norm(v2))

[-3.18715833 13.26330681 -1.85654086]
[2. 1. 5.]


In [17]:
def drawAxis():
    xId = pybullet.addUserDebugLine([0, 0, 0], [10, 0, 0], lineColorRGB=[1, 0, 0], lineWidth=2)
    yId = pybullet.addUserDebugLine([0, 0, 0], [0, 5, 0], lineColorRGB=[0, 1, 0], lineWidth=2)
    zId = pybullet.addUserDebugLine([0, 0, 0], [0, 0, 15], lineColorRGB=[0, 0, 1], lineWidth=2)
    return xId, yId, zId

def drawLine2Joint(df, j1, j2, frame, color):
    return pybullet.addUserDebugLine(getJointPos(df, j1, frame), getJointPos(df, j2, frame), lineColorRGB=color, lineWidth=5)

def getVector2Parts(parts, p1, p2):
    return parts[p2].get_position() - parts[p1].get_position()

def getVector2Joint(df, j1, j2, frame):
    return getJointPos(df, j2, frame) - getJointPos(df, j1, frame)

def angle2Vec(v1, v2):
    return np.arccos(v1.dot(v2)/(np.linalg.norm(v1) * np.linalg.norm(v2)))

def drawLine2Parts(env, p1, p2, color):
    delta = np.array([1, 0, 0])
    return pybullet.addUserDebugLine(env.parts[p1].get_position() + delta, env.parts[p1].get_position() + delta, lineColorRGB=color, lineWidth=5)

def drawLine(c1, c2, color):
    return pybullet.addUserDebugLine(c1, c2, lineColorRGB=color, lineWidth=5)

def draw2(p, color):
    return drawLine([-3, 0, 0], env.flat_env.parts[p].get_position(), color)

In [54]:
env.close()

In [25]:
env = make_env(ENV)

In [26]:
env.render()
observation = env.resetFromFrame(10)
pybullet.removeAllUserDebugItems()
drawAxis()

(0, 1, 2)

In [101]:
observation = env.resetFromFrame(30)

In [53]:
basePos = env.flat_env.parts['lwaist'].get_position()
pybullet.removeAllUserDebugItems()

target_pos = np.array([1, -1, 0])
drawLine([0, 0, 0], target_pos, [0, 0, 0])
r = R.from_euler('z', np.arctan2(target_pos[1], target_pos[0]))
env.flat_env.parts['torso'].reset_orientation(r.as_quat())

vll = basePos + r.apply(getNormalJointPos(norm_df, 'LeftLeg', 10))
vlf = basePos + r.apply(getNormalJointPos(norm_df, 'LeftFoot', 10))
drawLine(basePos, vll, [1, 0, 0])
drawLine(basePos, vlf, [0, 1, 0])

vrl = basePos + r.apply(getNormalJointPos(norm_df, 'RightLeg', 10))
vrf = basePos + r.apply(getNormalJointPos(norm_df, 'RightFoot', 10))
drawLine(basePos, vrl, [0, 0, 1])
drawLine(basePos, vrf, [1, 1, 1])

4

In [63]:
np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])[2:2+6]

array([2, 3, 4, 5, 6, 7])

In [103]:
v1 = env.flat_env.parts['left_foot'].get_position() 
v2 = basePos + getNormalJointPos(norm_df, 'LeftFoot', 10)

In [104]:
drawLine([0, 0, 0], v1, [0, 0, 1])
drawLine([0, 0, 0], v2, [0, 1, 0])
drawLine(v1, v2, [1, 1, 1])

4

In [105]:
np.linalg.norm(v2 - v1)

0.23758328596822187

In [65]:
drawLine(v1, v2, [1, 1, 1])

4

In [26]:
env.resetFromFrame(50)

array([ 3.4999999e-01, -1.3717048e-05,  1.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
        7.1336195e-02,  0.0000000e+00,  5.1822627e-01,  0.0000000e+00,
        1.2700889e-02,  0.0000000e+00,  1.1723903e+00,  0.0000000e+00,
        4.3820977e-01,  0.0000000e+00,  4.4546205e-01,  0.0000000e+00,
        5.5154860e-01,  0.0000000e+00,  9.6816510e-02,  0.0000000e+00,
        3.9674813e-01,  0.0000000e+00,  4.4077528e-01,  0.0000000e+00,
        1.2652647e-01,  0.0000000e+00,  1.2998933e-01,  0.0000000e+00,
        1.6158226e-01,  0.0000000e+00,  2.4842025e-01,  0.0000000e+00,
       -1.5762837e-01,  0.0000000e+00, -2.1543621e-01,  0.0000000e+00,
        2.8703377e-01,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
      dtype=float32)

In [29]:
env.close()

In [30]:
# pybullet.removeAllUserDebugItems()
# idx = 100
# v1 = getVector2Parts(env.flat_env.parts, 'link0_15', 'link0_19')
# v2 = getVector2Joint(bvh_pos, 'LeftUpLeg', 'LeftLeg', idx)

# r = rotFrom2Vec(v1, v2)
# rE = r.as_euler('xyz')
# print(rE)

# jx = env.flat_env.jdict['left_hip_x'].get_position()
# jy = env.flat_env.jdict['left_hip_y'].get_position()
# jz = env.flat_env.jdict['left_hip_z'].get_position()

# env.flat_env.jdict['left_hip_x'].set_state(jx - rE[0], 0)
# env.flat_env.jdict['left_hip_y'].set_state(jy + rE[1], 0)
# env.flat_env.jdict['left_hip_z'].set_state(jz - rE[2], 0)

# v11 = env.flat_env.parts['link0_15'].get_position()
# v12 = env.flat_env.parts['link0_19'].get_position()

# v21 = getJointPos(bvh_pos, 'LeftUpLeg', idx)
# v22 = getJointPos(bvh_pos, 'LeftLeg', idx)

# drawLine([-3, 0, 0], v11, [1, 1, 1])
# drawLine([-3, 0, 0], v12, [0, 0, 1])
# drawLine(v21, v21 + (v12- v11), [1, 1, 1])

# drawLine([-3, 0, 0], v21, [1, 0, 0])
# drawLine([-3, 0, 0], v22, [0, 1, 0])
# drawLine(v21, v22, [1, 1, 0])

In [68]:
def getRot(parts, l1, l2, df, j1, j2, frame):
    v1 = getVector2Parts(parts, l1, l2)
    v2 = getVector2Joint(df, j1, j2, frame)
    return rotFrom2Vec(v1, v2).as_euler('xyz')

def moveJoint(l1, l2, j1, j2, frame, part_name, weight, index):
    rot = getRot(env.flat_env.parts, l1, l2, bvh_pos, j1, j2, frame)
    cur_pos = [env.flat_env.jdict[p].get_position() for p in part_name]
    
    pos = []
    for i, idx in enumerate(index):
        joint_pos = cur_pos[i] + rot[idx] * weight[i]
        env.flat_env.jdict[part_name[i]].set_state(joint_pos, 0)
        pos.append(joint_pos)
    return pos

def setJoint(frame):
    rightHipX, rightHipY, rightHipZ = moveJoint('link0_8', 'link0_12', 'RightUpLeg', 'RightLeg', frame, ['right_hip_x', 'right_hip_y', 'right_hip_z'], [1, 1, 1], [0, 1, 2])
    rightKnee = moveJoint('link0_12', 'right_foot', 'RightLeg', 'RightFoot', frame, ['right_knee'], [-1], [1])[0]

    leftHipX, leftHipY,leftHipZ = moveJoint('link0_15', 'link0_19', 'LeftUpLeg', 'LeftLeg', frame, ['left_hip_x', 'left_hip_y', 'left_hip_z'], [-1, 1, -1], [0, 1, 2])
    leftKnee = moveJoint('link0_19', 'left_foot', 'LeftLeg', 'LeftFoot', frame, ['left_knee'], [-1], [1])[0]
    return [rightHipX, rightHipY, rightHipZ, rightKnee, leftHipX, leftHipY, leftHipZ, leftKnee]

In [95]:
env = make_env(ENV)
env.render()
observation = env.resetNonFrame()
pybullet.removeAllUserDebugItems()
drawAxis()

joint_data = []
for i in range(1, 100):
    joint_data.append(setJoint(i))
    time.sleep(1.0/20)
env.close()

In [92]:
joint_df = pd.DataFrame(joint_data, columns=['rightHipX', 'rightHipY', 'rightHipZ', 'rightKnee', 'leftHipX', 'leftHipY', 'leftHipZ', 'leftKnee'])

In [94]:
joint_df

,rightHipX,rightHipY,rightHipZ,rightKnee,leftHipX,leftHipY,leftHipZ,leftKnee
0,0.064196,-0.599307,0.036584,-1.028779,0.030262,-0.107818,0.067969,-0.536350
1,0.074723,-0.612231,0.042783,-1.000192,0.008741,-0.085250,0.068484,-0.527270
2,0.077187,-0.624273,0.043695,-0.954795,0.008774,-0.055518,0.070853,-0.502908
3,0.080630,-0.633907,0.045440,-0.904467,0.010291,-0.029381,0.072894,-0.483312
4,0.084916,-0.637582,0.048192,-0.846201,0.010385,-0.006985,0.074557,-0.466005
...,...,...,...,...,...,...,...,...
94,0.071359,0.108791,0.200814,-0.905737,-0.104117,-0.637404,0.149173,-0.729953
95,0.064254,0.078321,0.200434,-0.962352,-0.102162,-0.622116,0.152206,-0.738649
96,0.056748,0.045629,0.199883,-1.016857,-0.099599,-0.605594,0.155717,-0.742837
97,0.049528,0.011006,0.199086,-1.068893,-0.096813,-0.587826,0.159440,-0.742774


In [117]:
joint_df.to_csv('Joints CSV/walk8_1JointPos.csv', index=False)

In [35]:
# draw2('link0_8', [1, 0, 0]) # Kanan atas
# draw2('link0_12', [1, 0, 0]) # Kanan lutut
# draw2('right_foot', [1, 0, 0]) # Kanan telapak
# draw2('link0_15', [0, 1, 0]) # Kiri atas
# draw2('link0_19', [0, 1, 0]) # Kiri lutut
# draw2('left_foot', [0, 1, 0]) # Kiri telapak

In [67]:
pybullet.removeAllUserDebugItems()

In [146]:
drawLine2Parts(env.flat_env, 'pelvis', 'left_thigh', [1, 1, 1])
drawLine2Parts(env.flat_env, 'left_thigh', 'left_shin', [1, 1, 1])
drawLine2Parts(env.flat_env, 'left_shin', 'left_foot', [1, 1, 1])

drawLine2Parts(env.flat_env, 'pelvis', 'right_thigh', [0, 0, 1])
drawLine2Parts(env.flat_env, 'right_thigh', 'right_shin', [0, 0, 1])
drawLine2Parts(env.flat_env, 'right_shin', 'right_foot', [0, 0, 1])

5

In [36]:
for i in range(1, 100):
    drawLine2Joint(bvh_pos, 'Hips', 'LeftUpLeg', i, [1, 0, 0])
    drawLine2Joint(bvh_pos, 'LeftUpLeg', 'LeftLeg', i, [1, 0, 0])
    drawLine2Joint(bvh_pos, 'LeftLeg', 'LeftFoot', i, [1, 0, 0])
    
    drawLine2Joint(bvh_pos, 'Hips', 'RightUpLeg', i, [0, 1, 0])
    drawLine2Joint(bvh_pos, 'RightUpLeg', 'RightLeg', i, [0, 1, 0])
    drawLine2Joint(bvh_pos, 'RightLeg', 'RightFoot', i, [0, 1, 0])
    time.sleep(1.0/30)
    pybullet.removeAllUserDebugItems()

In [109]:
def setEnv(df):
    env.flat_env.jdict['right_knee'].set_state(df['rightKnee'], 0)

    env.flat_env.jdict['right_hip_x'].set_state(df['rightHipX'], 0)
    env.flat_env.jdict['right_hip_y'].set_state(df['rightHipY'], 0)
    env.flat_env.jdict['right_hip_z'].set_state(df['rightHipZ'], 0)
    
    env.flat_env.jdict['left_knee'].set_state(df['leftKnee'], 0)

    env.flat_env.jdict['left_hip_x'].set_state(df['leftHipX'], 0)
    env.flat_env.jdict['left_hip_y'].set_state(df['leftHipY'], 0)
    env.flat_env.jdict['left_hip_z'].set_state(df['leftHipZ'], 0)

setEnv(joint_df.iloc[0])

In [111]:
setEnv(joint_df.iloc[80])

In [114]:
for i in range(10):
    observation = env.resetNonFrame()
    idx = 0
    while(idx < len(joint_df)):
        # action = agent.compute_action(observation)
        # observation, reward, done, info = env.step(action)
        setEnv(joint_df.iloc[idx])
        idx += 1
        time.sleep(1.0/30)

In [115]:
env.close()